In [6]:
# importing all the required libraries

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.support.ui import Select

import xlsxwriter
import time

In [7]:
# adding options

chromeOptions = Options()
chromeOptions.add_argument('--kiosk')
chromeOptions.add_argument('blink-settings=imagesEnabled=false')
chromeOptions.page_load_strategy = 'normal'

In [8]:
# opening the webpage

driver = webdriver.Chrome('../chromedriver/chromedriver', options=chromeOptions)

In [4]:
driver.get('https://www.producthunt.com')

In [10]:
driver.close()

# Extract Product Hunt listings
### Steps:
1. Scroll down the page
2. Click all the show more button (to be able to scrape all data)
3. Scrape the dates (for reference: to see which timeframe was scraped)
4. Get all the cards
5. Loop the cards then scrape all the necessary data(then add it into a list):
    - Product Name
    - Description
    - Tags
    - Votes
    - href (to get the URL since it doesn't exist in the home page)  
6. Loop the list and go to each href then scrape the website to get the URL.
7. Save the lists with complete information to an excel file.

# Step 1: Scroll down

In [5]:
for i in range(300): # scrolling down the page
    print(i, driver.execute_script("return document.body.scrollHeight"))
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1) # give the browser some time to load

0 3528
1 3921
2 3921
3 6137
4 7465
5 8793
6 10121
7 10514
8 11449
9 11862
10 12797
11 13190
12 14125
13 15453
14 16781
15 18109
16 19457
17 20785
18 22133
19 23481
20 24809
21 26137
22 27465
23 28793
24 30121
25 31449
26 32777
27 34105
28 35433
29 36761
30 38089
31 39417
32 40745
33 41138
34 42093
35 43421
36 44769
37 46097
38 47425
39 48753
40 50101
41 51429
42 52757
43 54085
44 55413
45 56741
46 58069
47 59397
48 60725
49 62073
50 63401
51 64729
52 66057
53 67385
54 68713
55 70041
56 71413
57 72741
58 74069
59 75397
60 76725
61 78073
62 79401
63 80729
64 82057
65 83385
66 84713
67 86041
68 87369
69 88697
70 90025
71 91353
72 92681
73 94009
74 95357
75 96685
76 98013
77 99341
78 100669
79 101997
80 103325
81 104653
82 105981
83 107329
84 108657
85 109985
86 111313
87 112641
88 113969
89 115317
90 116645
91 117973
92 119301
93 120629
94 121957
95 123285
96 124633
97 125961
98 127289
99 128617
100 129010
101 129945
102 131273
103 132621
104 133949
105 135297
106 136625
107 137953
108 13

In [30]:
print(driver.current_url)

https://www.producthunt.com/


# Step 2: Click all show more buttons

In [8]:
# click all show more buttons
show_more = driver.find_elements_by_class_name('showHiddenPosts_fbc1f')
print(len(show_more))
for show in show_more:
    show.click()

192


# Step 3: Take all dates and print it

In [7]:
# get all dates (for reference as to when to when we got the dates.)
get_all_date = driver.find_element_by_tag_name('main').find_elements_by_class_name('header_fb2c3')
print(len(get_all_date))
for date in get_all_date: 
    print(date.text)

296
Upcoming
powered by Ship
Today
POPULARNEWEST
Best products last week
Yesterday
April 18th
April 17th
April 16th
April 15th
April 14th
April 13th
April 12th
April 11th
April 10th
April 9th
April 8th
April 7th
April 6th
April 5th
April 4th
April 3rd
April 2nd
April 1st
March 31st
March 30th
March 29th
March 28th
March 27th
March 26th
March 25th
March 24th
March 23rd
March 22nd
March 21st
March 20th
March 19th
March 18th
March 17th
March 16th
March 15th
March 14th
March 13th
March 12th
March 11th
March 10th
March 9th
March 8th
March 7th
March 6th
March 5th
March 4th
March 3rd
March 2nd
March 1st
February 29th
February 28th
February 27th
February 26th
February 25th
February 24th
February 23rd
February 22nd
February 21st
February 20th
February 19th
February 18th
February 17th
February 16th
February 15th
February 14th
February 13th
February 12th
February 11th
February 10th
February 9th
February 8th
February 7th
February 6th
February 5th
February 4th
February 3rd
February 2nd
February 1st

# Step 4: Find all cards

In [9]:
cards = driver.find_elements_by_class_name('item_54fdd') # getting all cards from classname
print(len(cards))
products = []

6471


# Step 5: Loop cards and extract data

In [10]:
for i, card in enumerate(cards):
    try:
        title = card.find_element_by_tag_name('h3').text # getting title from h3 in each li
        description = card.find_element_by_tag_name('p').text # getting description from p in each li
        vote = card.find_element_by_class_name('voteButtonWrap_4c515').find_element_by_tag_name('span').text # getting vote from classname then span
        href = card.find_element_by_tag_name('a').get_attribute('href') # get href through anchor tag
        products.append({'product': title, 'description': description, 'tag': [], 'vote': vote, 'url': None, 'href': href})
    except:
        print('error', i)

error 6311


In [12]:
print(len(products))
# for i in products:
#     print(i)

6469


# Step 6: Go to each website and get the URL and tags

In [16]:
for product in products:
    if(product['url'] == None):
        driver.get(product['href'])
        try:
            driver.find_element_by_class_name('more_2e78b').click()
        except:
            print('No clickable tag')
        for tag in driver.find_elements_by_class_name('button_53e93'):
            product['tag'].append(tag.find_element_by_tag_name('a').get_attribute('title'))
        driver.get(driver.find_element_by_class_name('link_9bebc').get_attribute('href'))
        product['url'] = driver.current_url
        print(product)

[{'product': 'Good Books', 'description': 'Largest curated collection of 6,500+ book recommendations.', 'tag': ['Productivity', 'History books'], 'vote': 685, 'url': 'https://www.goodbooks.io/', 'href': 'https://www.producthunt.com/posts/good-books'}, {'product': 'SpatialChat', 'description': 'Your virtual party for up to 50 guests', 'tag': ['User experience', 'Web app', 'Video streaming'], 'vote': 400, 'url': 'https://spatial.chat/', 'href': 'https://www.producthunt.com/posts/spatialchat'}, {'product': 'Snack', 'description': 'Have spontaneous, time-restricted video conversations.', 'tag': ['Slack', 'Bots'], 'vote': 274, 'url': 'https://aboutsnack.com/', 'href': 'https://www.producthunt.com/posts/snack-4'}, {'product': '8base', 'description': 'The GraphQL Backend that harnesses the full power of AWS.', 'tag': ['nan'], 'vote': 669, 'url': 'https://www.8base.com/producthunt', 'href': 'https://www.producthunt.com/r/p/171794?ref=home&promo_id=10852'}, {'product': 'Payment Pages by involve

In [17]:
count = 0
for (i, product) in enumerate(products):
    if(product['url'] == None):
        print(product)
        count += 1
#         del products[i]
#         break
print(count)

0


# Fixing Datas

In [9]:
# url == href
for product in products:
    if(product['url'] == product['href']):
        driver.get(product['href'])
        driver.get(driver.find_element_by_class_name('link_9bebc').get_attribute('href'))
        product['url'] = driver.current_url
        print(product)

{'product': 'Secure Send for Slack', 'description': 'Send files securely without leaving your Slack channel!', 'tag': ['Privacy', 'Slack', 'Productivity'], 'vote': 31, 'url': 'https://slack.securesend.quantfive.org/?ref=producthunt', 'href': 'https://www.producthunt.com/posts/secure-send-for-slack'}
{'product': 'Ratee', 'description': 'Get rated on your attractiveness', 'tag': ['Web App', 'Dating'], 'vote': 31, 'url': 'https://ratee.me/?ref=producthunt', 'href': 'https://www.producthunt.com/posts/ratee'}
{'product': 'Tablets', 'description': 'Start tracking and analyzing your substance use.', 'tag': ['Productivity', 'Analytics', 'Tech', 'Apple', 'iPhone', 'Health and Fitness'], 'vote': 146, 'url': 'https://apps.apple.com/app/id1460296357?ref=producthunt', 'href': 'https://www.producthunt.com/posts/tablets'}


KeyboardInterrupt: 

In [46]:
# products with trailing tags
for product in products:
    correctTags = True
    for tag in product['tag']:
        if('...' in tag):
            correctTags = False
            print(tag)
    if(not correctTags):
        driver.get(product['href'])
        product['tag'] = []
        try:
            driver.find_element_by_class_name('more_2e78b').click()
        except:
            print('No clickable tag')
        for tag in driver.find_elements_by_class_name('button_53e93'):
            product['tag'].append(tag.find_element_by_tag_name('a').get_attribute('title'))
        
        print(product)

In [23]:
print(len(products))

6470


# Step 7: Put data into excel file

In [13]:
workbook = xlsxwriter.Workbook('producthunt.xlsx')
worksheet = workbook.add_worksheet()
worksheet.set_column('A:D', 25)
worksheet.set_column('E:E', 70)
worksheet.write(0, 0, 'Product Name')
worksheet.write(0, 1, 'Description')
worksheet.write(0, 2, 'URL')
worksheet.write(0, 3, 'ProductHunt URL')
worksheet.write(0, 4, 'Tag')
worksheet.write(0, 5, 'Votes')

row = 1
for product in products:
    worksheet.write(row, 0, product['product'])
    worksheet.write(row, 1, product['description'])
    worksheet.write(row, 2, product['url'].split('?')[0])
    worksheet.write(row, 3, product['href'])
    if(product['tag'][0] == 'nan'):
        worksheet.write(row, 4, ','.join(product['tag'][1:]))
    else:
        worksheet.write(row, 4, ','.join(product['tag']))
    worksheet.write(row, 5, int(product['vote']))
    row += 1
    
workbook.close()

# Continuation
Import from excel and continue getting the urls

In [1]:
import pandas as pd
import numpy as np

In [13]:
df = pd.read_excel('producthunt.xlsx')
products = []
print(df)

                     Product Name  \
0                      Good Books   
1                     SpatialChat   
2                           Snack   
3                           8base   
4     Payment Pages by involve.me   
...                           ...   
6464             The New Girlboss   
6465                      ProGram   
6466                    Adv3nture   
6467                 Squirrly SPY   
6468                     GANpaint   

                                            Description  \
0     Largest curated collection of 6,500+ book reco...   
1                Your virtual party for up to 50 guests   
2     Have spontaneous, time-restricted video conver...   
3     The GraphQL Backend that harnesses the full po...   
4     Collect Stripe & PayPal payments online with c...   
...                                                 ...   
6464    A new professional network for millennial women   
6465     CLI based, self-hosted Instagram-esque gallery   
6466                   3

In [14]:
for product in df.iloc:
    products.append({'product': str(product['Product Name']), 'description': str(product['Description']), 'tag': str(product['Tag']).split(','), 'vote': int(product['Votes']), 'url': str(product['URL']), 'href': str(product['ProductHunt URL'])})


In [18]:
print(len(products))
for product in products:
    print(product)

6469
{'product': 'Good Books', 'description': 'Largest curated collection of 6,500+ book recommendations.', 'tag': ['Productivity', 'History books'], 'vote': 685, 'url': 'https://www.goodbooks.io/', 'href': 'https://www.producthunt.com/posts/good-books'}
{'product': 'SpatialChat', 'description': 'Your virtual party for up to 50 guests', 'tag': ['User experience', 'Web app', 'Video streaming'], 'vote': 400, 'url': 'https://spatial.chat/', 'href': 'https://www.producthunt.com/posts/spatialchat'}
{'product': 'Snack', 'description': 'Have spontaneous, time-restricted video conversations.', 'tag': ['Slack', 'Bots'], 'vote': 274, 'url': 'https://aboutsnack.com/', 'href': 'https://www.producthunt.com/posts/snack-4'}
{'product': '8base', 'description': 'The GraphQL Backend that harnesses the full power of AWS.', 'tag': ['nan'], 'vote': 669, 'url': 'https://www.8base.com/producthunt', 'href': 'https://www.producthunt.com/r/p/171794?ref=home&promo_id=10852'}
{'product': 'Payment Pages by involve

{'product': 'Spory', 'description': 'AI assistant to help you organize your sport activities', 'tag': ['Artificial Intelligence', 'Bots', 'Productivity', 'Sports'], 'vote': 293, 'url': 'https://spory.codes/', 'href': 'https://www.producthunt.com/posts/spory'}
{'product': "Alto's Odyssey for Mac", 'description': 'Discover the endless desert', 'tag': ['Platformers', 'Adventure Games', 'Games', 'iPhone', 'Mac'], 'vote': 179, 'url': 'http://altosodyssey.com/', 'href': 'https://www.producthunt.com/posts/alto-s-odyssey-for-mac'}
{'product': 'Bet On Weather', 'description': 'Bet against your local meteorologist. Make money.', 'tag': ['nan'], 'vote': 131, 'url': 'https://betonweather.io/', 'href': 'https://www.producthunt.com/posts/bet-on-weather'}
{'product': 'FindMeGigs', 'description': 'A bot to find gigs and leads for you on Twitter ✨', 'tag': ['Freelance', 'Robots', 'Hiring and Recruiting', 'Productivity'], 'vote': 100, 'url': 'https://findmegigs.com/', 'href': 'https://www.producthunt.co

{'product': 'Refined GitHub Notifications', 'description': 'UI improvements for the new GitHub Notifications page', 'tag': ['GitHub', 'Productivity', 'Open Source'], 'vote': 36, 'url': 'https://chrome.google.com/webstore/detail/refined-github-notificati/apeppglkpocpebbdnocaogepmghkecic', 'href': 'https://www.producthunt.com/posts/refined-github-notifications'}
{'product': 'Generative.Photos', 'description': '25k diverse & customizable free stock photos generated by AI', 'tag': ['Photography', 'Artificial Intelligence', 'Design Tools', 'Marketing'], 'vote': 1018, 'url': 'https://www.generative.photos/', 'href': 'https://www.producthunt.com/posts/generative-photos'}
{'product': 'IndieMailer', 'description': 'Community of paid newsletter creators 💌', 'tag': ['Marketing', 'Tech', 'Email Newsletters', 'Email Marketing'], 'vote': 674, 'url': 'https://indiemailer.com/', 'href': 'https://www.producthunt.com/posts/indiemailer'}
{'product': 'mynameis', 'description': 'Ridiculously expensive vide

{'product': 'Revitalize by Pragli', 'description': 'Curated mental health resources for makers', 'tag': ['Tech', 'Productivity', 'Meditation'], 'vote': 66, 'url': 'https://pragli.com/revitalize', 'href': 'https://www.producthunt.com/posts/revitalize-by-pragli'}
{'product': 'Predator Thronos Air', 'description': 'A $14,000 gaming chair from Acer', 'tag': ['Home', 'Games'], 'vote': 28, 'url': 'https://community.acer.com/en/discussion/576165/next-acer-ifa-2019-predator-triton-300-predator-thronos-air-planet-9', 'href': 'https://www.producthunt.com/posts/predator-thronos-air'}
{'product': 'localsnear.me', 'description': 'A simple app to find your next favorite coffee shop', 'tag': ['Coffee', 'Maps'], 'vote': 30, 'url': 'https://localsnear.me/', 'href': 'https://www.producthunt.com/posts/localsnear-me'}
{'product': 'GoGaga', 'description': 'A dating app that connects you to friends of friends', 'tag': ['User Experience', 'Facebook', 'Dating', 'LinkedIn', 'Tech', 'Android', 'iPhone'], 'vote'

In [19]:
# get product urls and tags
for product in products:
    if(product['url'] == 'nan'):
        driver.get(product['href'])
        try:
            driver.find_element_by_class_name('more_2e78b').click()
        except:
            print('No clickable tag')
        for tag in driver.find_elements_by_class_name('button_53e93'):
            product['tag'].append(tag.find_element_by_tag_name('a').get_attribute('title'))
        driver.get(driver.find_element_by_class_name('link_9bebc').get_attribute('href'))
        product['url'] = driver.current_url
        print(product)

No clickable tag
{'product': 'Easymove', 'description': 'Uber for large item delivery', 'tag': ['Home', 'Home'], 'vote': 214, 'url': 'https://geteasymove.com/?ref=producthunt', 'href': 'https://www.producthunt.com/posts/easymove-1'}
No clickable tag
{'product': 'Mastermind', 'description': 'Build habits by using the addictive powers of social media.', 'tag': ['nan', 'Health and Fitness', 'Productivity'], 'vote': 170, 'url': 'http://mastermindapp.co/?ref=producthunt', 'href': 'https://www.producthunt.com/posts/mastermind'}
{'product': 'Rambox Pro', 'description': 'The best way to organize your workspace. 💪🏼🚀', 'tag': ['nan', 'Mac', 'Web App', 'Email', 'Productivity', 'Messaging', 'SaaS', 'Tech', 'Linux', 'Windows'], 'vote': 352, 'url': 'https://rambox.pro/?ref=producthunt', 'href': 'https://www.producthunt.com/posts/rambox-pro'}
No clickable tag
{'product': 'Helios Touch Lighting', 'description': 'Swipe for light. Make walls a glowing canvas', 'tag': ['nan', 'Tech'], 'vote': 116, 'url':

{'product': 'Travellar', 'description': 'A social travel app', 'tag': ['nan', 'Travel', 'Tech', 'Android', 'iPhone'], 'vote': 125, 'url': 'https://www.travellar.app/?ref=producthunt', 'href': 'https://www.producthunt.com/posts/travellar-2'}
No clickable tag
{'product': 'Mahmee', 'description': 'A digital support network for new moms', 'tag': ['nan', 'Home', 'Kids'], 'vote': 72, 'url': 'https://www.mahmee.com/?ref=producthunt', 'href': 'https://www.producthunt.com/posts/mahmee-2'}
No clickable tag
{'product': 'Agaze', 'description': 'Simple, encrypted website analytics', 'tag': ['nan', 'Analytics', 'Blockstack'], 'vote': 74, 'url': 'https://agaze.co/?ref=producthunt', 'href': 'https://www.producthunt.com/posts/agaze'}
No clickable tag
{'product': 'Timehacker', 'description': 'Automatic time tracking for self-improvement', 'tag': ['nan', 'Productivity', 'Time Tracking'], 'vote': 85, 'url': 'https://timehacker.app/?ref=producthunt', 'href': 'https://www.producthunt.com/posts/timehacker'}


{'product': 'Positiive', 'description': 'Share and see what people are grateful for 🤗☀️', 'tag': ['nan', 'Social Media Tools', 'Web App', 'Health and Fitness'], 'vote': 168, 'url': 'https://positiive.app/?ref=producthunt#/', 'href': 'https://www.producthunt.com/posts/positiive'}
{'product': 'Beelinguapp', 'description': 'Learn Languages with audiobooks and music', 'tag': ['nan', 'Productivity', 'Tech', 'Android', 'iPhone'], 'vote': 154, 'url': 'https://www.beelinguapp.com/?ref=producthunt', 'href': 'https://www.producthunt.com/posts/beelinguapp'}
{'product': 'Dev Zines!', 'description': 'A list of zines for developers & resources to make your own', 'tag': ['nan', 'Tech', 'Books', 'Education', 'Developer Tools'], 'vote': 125, 'url': 'https://zines.dev/?ref=producthunt', 'href': 'https://www.producthunt.com/posts/dev-zines'}
No clickable tag
{'product': 'JobCull', 'description': 'Save and track your applied jobs across the internet', 'tag': ['nan', 'Productivity'], 'vote': 98, 'url': 'ht

{'product': 'Quadruple Cube Jump', 'description': 'Flip through obstacles', 'tag': ['nan', 'Indie Games', 'Free Games', 'Retro Games', 'Games', 'iPhone', 'Adventure Games'], 'vote': 29, 'url': 'https://apps.apple.com/us/app/quadruple-cube-jump-3d/id1452119487?ref=producthunt', 'href': 'https://www.producthunt.com/posts/quadruple-cube-jump'}
{'product': 'Framer Motion', 'description': 'A truly simple production-ready React animation library', 'tag': ['nan', 'Developer Tools', 'Tech', 'Productivity', 'User Experience'], 'vote': 885, 'url': 'https://www.framer.com/motion/?ref=producthunt', 'href': 'https://www.producthunt.com/posts/framer-motion'}
{'product': 'BentenSound', 'description': 'Free stock music for your videos. Unsplash for music.', 'tag': ['nan', 'YouTube', 'Tech', 'Blockstack', 'Web App', 'Music'], 'vote': 1029, 'url': 'https://bentensound.com/?ref=producthunt', 'href': 'https://www.producthunt.com/posts/bentensound'}
No clickable tag
{'product': '6nomads', 'description': 'F

{'product': 'Bunch.ai', 'description': 'Culture analytics for teams that use Slack - Try it free!', 'tag': ['nan', 'SaaS', 'Artificial Intelligence', 'Slack', 'Analytics'], 'vote': 424, 'url': 'https://bunch.ai/?utm_source=ph&utm_medium=submit&utm_campaign=betab&ref=producthunt', 'href': 'https://www.producthunt.com/posts/bunch-ai'}
No clickable tag
{'product': 'Nintendo Switch Lite', 'description': 'A new compact version of the Nintendo Switch', 'tag': ['nan', 'Games'], 'vote': 291, 'url': 'https://www.nintendo.com/switch/lite/?ref=producthunt', 'href': 'https://www.producthunt.com/posts/nintendo-switch-lite'}
{'product': 'Insense Barter with Influencers', 'description': 'Strike barter deals with Instagram Influencers', 'tag': ['nan', 'Marketing', 'Tech', 'Instagram', 'Social Media Tools'], 'vote': 333, 'url': 'https://insense.pro/barter-campaigns/?ref=producthunt', 'href': 'https://www.producthunt.com/posts/insense-barter-with-influencers'}
{'product': 'WebARX', 'description': 'Prote

...

No clickable tag
{'product': 'Nexo', 'description': 'Banking on Crypto — Borrow. Earn. Pay. Invest.', 'tag': ['nan', 'Fintech', 'Crypto'], 'vote': 595, 'url': 'https://nexo.io/?ref=producthunt', 'href': 'https://www.producthunt.com/posts/nexo'}
{'product': 'Reelshot', 'description': 'Edit videos in real time as you shoot them 🎬', 'tag': ['nan', 'Social Media Tools', 'YouTube', 'Tech', 'iPhone', 'Video Streaming'], 'vote': 176, 'url': 'https://apps.apple.com/bh/app/reelshot/id1449364632?ref=producthunt', 'href': 'https://www.producthunt.com/posts/reelshot'}
{'product': 'JustSnake', 'description': 'Snake on a ✈️ blockchain', 'tag': ['nan', 'Games', 'Blockstack', 'Funny', 'Crypto'], 'vote': 181, 'url': 'https://justsnake.live/?ref=producthunt', 'href': 'https://www.producthunt.com/posts/justsnake'}
{'product': 'Backstory for Facebook', 'description': 'Private notes to remember and search for FB friends', 'tag': ['nan', 'Social Media Tools', 'Software Engineering', 'Tech', 'Chrome Extensio

{'product': 'Blindy', 'description': 'A hard platformer game to test your memory', 'tag': ['nan', 'Indie Games', 'Games', 'Android', 'Platformers'], 'vote': 85, 'url': 'https://play.google.com/store/apps/details?id=com.radoslawfelich.blindy2dplatformer&ref=producthunt', 'href': 'https://www.producthunt.com/posts/blindy-2'}
{'product': 'OutOfTheLoop', 'description': 'Stop spending hours together on YouTube', 'tag': ['nan', 'User Experience', 'YouTube', 'Chrome Extensions', 'Productivity'], 'vote': 40, 'url': 'https://chrome.google.com/webstore/detail/outoftheloop/heefnbchlijbbgpbpidiieoldoopagac?ref=producthunt', 'href': 'https://www.producthunt.com/posts/outoftheloop'}
{'product': 'Breathe2', 'description': 'Pre-installed workouts & option to create your own, for free', 'tag': ['nan', 'Health and Fitness', 'User Experience', 'iPhone', 'Apple Watch'], 'vote': 39, 'url': 'https://apps.apple.com/us/app/breathe2/id1456731249?l=de&ref=producthunt', 'href': 'https://www.producthunt.com/posts

No clickable tag
{'product': 'Satback', 'description': 'Pay your customers for honest feedback', 'tag': ['nan', 'Customer communication', 'Blockstack'], 'vote': 47, 'url': 'https://satback.tk/?ref=producthunt', 'href': 'https://www.producthunt.com/posts/satback'}
No clickable tag
{'product': 'SkyAlliance Travel', 'description': 'Compare cheap flight + hotel prices across the world.', 'tag': ['nan', 'iPhone', 'Travel'], 'vote': 42, 'url': 'https://www.skyalliance.co.uk/?ref=producthunt', 'href': 'https://www.producthunt.com/posts/skyalliance-travel'}
No clickable tag
{'product': 'Contalist for Trello', 'description': 'Work with your contacts without leaving a Trello card', 'tag': ['nan', 'Productivity'], 'vote': 35, 'url': 'https://trello.com/power-ups/5b97e9450f54a3562b2ff848?ref=producthunt', 'href': 'https://www.producthunt.com/posts/contalist-for-trello'}
{'product': 'Blockcharity', 'description': 'Charitable deposits with Bitcoin', 'tag': ['nan', 'Blockstack', 'Charity and Giving',

{'product': 'Klaus', 'description': 'Conversation review tool for support teams', 'tag': ['nan', 'Customer communication', 'Tech', 'Web App', 'Productivity'], 'vote': 345, 'url': 'https://www.klausapp.com/?ref=producthunt', 'href': 'https://www.producthunt.com/posts/klaus-2'}
No clickable tag
{'product': 'Clever Grid', 'description': 'Easy to use and fairly priced GPUs for Machine Learning', 'tag': ['nan', 'Developer Tools', 'Artificial Intelligence'], 'vote': 226, 'url': 'https://www.clevergrid.io/?ref=producthunt', 'href': 'https://www.producthunt.com/posts/clever-grid'}
{'product': 'Tripetto for WordPress', 'description': 'Build engaging forms & surveys in just minutes', 'tag': ['nan', 'Privacy', 'Developer Tools', 'Customer communication', 'WordPress'], 'vote': 209, 'url': 'https://tripetto.com/wordpress/?ref=producthunt', 'href': 'https://www.producthunt.com/posts/tripetto-for-wordpress'}
{'product': 'ArtiBot AI - Free Chatbot for Websites', 'description': 'Create a free chatbot f

No clickable tag
{'product': 'Design Tools Weekly', 'description': 'Design tools & guides on using them, sent to your 📨', 'tag': ['nan', 'Design Tools', 'Email Newsletters'], 'vote': 312, 'url': 'https://flawlessapp.io/designtoolsweekly?ref=producthunt', 'href': 'https://www.producthunt.com/posts/design-tools-weekly'}
No clickable tag
{'product': 'Pipe.blue', 'description': 'The free link shortener for developers', 'tag': ['nan', 'Developer Tools', 'Tech'], 'vote': 111, 'url': 'https://pipe.blue/?ref=producthunt', 'href': 'https://www.producthunt.com/posts/pipe-blue'}
No clickable tag
{'product': 'Maker Goals', 'description': 'A new way to manage your Product Hunt goals', 'tag': ['nan', 'Web App', 'Product Hunt'], 'vote': 121, 'url': 'https://mkrgoals.com/?ref=producthunt', 'href': 'https://www.producthunt.com/posts/maker-goals'}
No clickable tag
{'product': 'Simple UTM Manager', 'description': 'Save and reuse your UTM campaign parameters for free', 'tag': ['nan', 'Productivity', 'Mark

In [4]:
count = 0
for product in products:
    if(product['url'] == 'nan'):
        print(product)
        count += 1
print(count)

{'product': 'Easymove', 'description': 'Uber for large item delivery', 'tag': ['Home'], 'vote': 214, 'url': 'nan', 'href': 'https://www.producthunt.com/posts/easymove-1'}
{'product': 'Mastermind', 'description': 'Build habits by using the addictive powers of social media.', 'tag': ['nan'], 'vote': 170, 'url': 'nan', 'href': 'https://www.producthunt.com/posts/mastermind'}
{'product': 'Rambox Pro', 'description': 'The best way to organize your workspace. 💪🏼🚀', 'tag': ['nan'], 'vote': 352, 'url': 'nan', 'href': 'https://www.producthunt.com/posts/rambox-pro'}
{'product': 'Helios Touch Lighting', 'description': 'Swipe for light. Make walls a glowing canvas', 'tag': ['nan'], 'vote': 116, 'url': 'nan', 'href': 'https://www.producthunt.com/posts/helios-touch-lighting'}
{'product': 'The Noonies', 'description': 'Tech Industry Awards by Hacker Noon', 'tag': ['nan'], 'vote': 149, 'url': 'nan', 'href': 'https://www.producthunt.com/posts/the-noonies'}
{'product': 'Gridea', 'description': 'A static 